In [2]:
# import all necessary libraries

import functions

from functions import mask_data

import geopandas as gpd
import xarray
import rioxarray
from shapely.geometry import mapping

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

import pandas as pd
from datetime import datetime

import os

from sklearn.metrics import mean_squared_error

import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mticker

from shapely.geometry import Point

In [3]:
def create_xarray(data,text):
    data_x = xarray.DataArray(data=data, coords={"time": full_time}, dims=["time"],name=text)
    return data_x


### Process GRACE

In [4]:
# import MDB and subregion boundaries 
MDB_shape = gpd.read_file('MDB_Regions/MDB_Whole.shp')
MDB_northeast_shape = gpd.read_file('MDB_Regions/MDB_Northeast_Region.shp')
MDB_northwest_shape = gpd.read_file('MDB_Regions/MDB_Northwest_Region.shp')
MDB_southeast_shape = gpd.read_file('MDB_Regions/MDB_Southeast_Region.shp')
MDB_southwest_shape = gpd.read_file('MDB_Regions/MDB_Southwest_Region.shp')
MDB_regions_combined = gpd.read_file('MDB_Regions/MDB_Regions_Combined.shp')

In [5]:
# Load GRACE data
mat_data = sio.loadmat('GRACE_Data/Monthly_TWS_over_Australia_mmH2O_360by720_200205_202404.mat')
lon = mat_data['lon']
lat = mat_data['lat']
tv = mat_data['tv']
TWS_origin = mat_data['TWS_origin']
TWS_downscaled = mat_data['TWS_downscaled']

In [6]:
# Extract year, month, and day columns
years = tv[:, 0]
months = tv[:, 1]
days = tv[:, 2]

# Create datetime objects
dates = np.array([datetime(year, month, day) for year, month, day in zip(years, months, days)])

In [7]:
# Convert the TWS_origin data into an xarray DataArray
TWS_origin_xr = xarray.DataArray(TWS_origin, dims=["latitude", "longitude", "time"],
                                 coords={"latitude": lat.flatten(), "longitude": lon.flatten(), "time": dates.flatten()})

# Convert the TWS_downscaled data into an xarray DataArray
TWS_downscaled_xr = xarray.DataArray(TWS_downscaled, dims=["latitude", "longitude", "time"],
                                 coords={"latitude": lat.flatten(), "longitude": lon.flatten(), "time": dates.flatten()})

In [8]:
# Define CRS consistently
TWS_origin_xr.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True);
TWS_origin_xr.rio.write_crs("epsg:4326", inplace=True);

TWS_downscaled_xr.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True);
TWS_downscaled_xr.rio.write_crs("epsg:4326", inplace=True);

In [9]:
# Process datasets
ne_GRACE_origin = functions.process_GRACE(TWS_origin_xr, MDB_northeast_shape)
nw_GRACE_origin = functions.process_GRACE(TWS_origin_xr, MDB_northwest_shape)
se_GRACE_origin = functions.process_GRACE(TWS_origin_xr, MDB_southeast_shape)
sw_GRACE_origin = functions.process_GRACE(TWS_origin_xr, MDB_southwest_shape)
MDB_GRACE_origin = functions.process_GRACE(TWS_origin_xr, MDB_shape)

ne_GRACE_downscaled = functions.process_GRACE(TWS_downscaled_xr, MDB_northeast_shape)
nw_GRACE_downscaled = functions.process_GRACE(TWS_downscaled_xr, MDB_northwest_shape)
se_GRACE_downscaled = functions.process_GRACE(TWS_downscaled_xr, MDB_southeast_shape)
sw_GRACE_downscaled = functions.process_GRACE(TWS_downscaled_xr, MDB_southwest_shape)
MDB_GRACE_downscaled = functions.process_GRACE(TWS_downscaled_xr, MDB_shape)

In [10]:
# Plot GRACE

# Plot GRACE observed Water Storage Over Time for MDB
plt.figure(figsize=(10, 6))
plt.plot(MDB_GRACE_downscaled.time, MDB_GRACE_downscaled, label='Terrestral Water Storage Anomaly (TWS_downscaled)', color='blue')
plt.plot(MDB_GRACE_origin.time, MDB_GRACE_origin, label='Terrestral Water Storage Anomaly (TWS_origin)', color='red')
plt.title('Terrestral Water Storage Anomaly in MDB from GRACE')
plt.ylabel('Terrestral Water Storage Anomaly (m)')
plt.legend()
plt.grid(True)
#plt.show()
plt.close()

In [12]:
# Plot GRACE observed Water Storage for all four sub-basins

fig, axs = plt.subplots(2, 2, figsize=(16, 12))
axs = axs.flatten()


# Plot 1 - NE
axs[0].plot(ne_GRACE_downscaled.time, ne_GRACE_downscaled, label='Terrestral Water Storage Anomaly (TWS_downscaled)', color='blue')
axs[0].plot(ne_GRACE_origin.time, ne_GRACE_origin, label='Terrestral Water Storage Anomaly (TWS_origin)', color='red')
axs[0].set_title('Terrestral Water Storage Anomaly in the Northeast MDB Region from GRACE')
axs[0].set_ylabel('Terrestral Water Storage Anomaly (m)')
axs[0].legend()
axs[0].grid(True)

# Plot 2 - NW
axs[1].plot(nw_GRACE_downscaled.time, nw_GRACE_downscaled, label='Terrestral Water Storage Anomaly (TWS_downscaled)', color='blue')
axs[1].plot(nw_GRACE_origin.time, nw_GRACE_origin, label='Terrestral Water Storage Anomaly (TWS_origin)', color='red')
axs[1].set_title('Terrestral Water Storage Anomaly in the Northwest MDB Region from GRACE')
axs[1].set_ylabel('Terrestral Water Storage Anomaly (m)')
axs[1].legend()
axs[1].grid(True)

# Plot 3 - SE
axs[2].plot(se_GRACE_downscaled.time, se_GRACE_downscaled, label='Terrestral Water Storage Anomaly (TWS_downscaled)', color='blue')
axs[2].plot(se_GRACE_origin.time, se_GRACE_origin, label='Terrestral Water Storage Anomaly (TWS_origin)', color='red')
axs[2].set_title('Terrestral Water Storage Anomaly in the Southeast MDB Region from GRACE')
axs[2].set_ylabel('Terrestral Water Storage Anomaly (m)')
axs[2].legend()
axs[2].grid(True)

# Plot 4 - SW
axs[3].plot(sw_GRACE_downscaled.time, sw_GRACE_downscaled, label='Terrestral Water Storage Anomaly (TWS_downscaled)', color='blue')
axs[3].plot(sw_GRACE_origin.time, sw_GRACE_origin, label='Terrestral Water Storage Anomaly (TWS_origin)', color='red')
axs[3].set_title('Terrestral Water Storage Anomaly in the Northeast MDB Region from GRACE')
axs[3].set_ylabel('Terrestral Water Storage Anomaly (m)')
axs[3].legend()
axs[3].grid(True)

#plt.show()
plt.close()

In [11]:
# Use Downscaled data

ne_GRACE = ne_GRACE_downscaled
nw_GRACE = nw_GRACE_downscaled
se_GRACE = se_GRACE_downscaled
sw_GRACE = sw_GRACE_downscaled
MDB_GRACE = MDB_GRACE_downscaled

In [12]:
# Calculate dTWS from aTWS

def dTWS (GRACE_data):
    dTWS = GRACE_data.diff(dim="time")
    dTWS["time"] = GRACE_data.time[1:]
    return dTWS

ne_GRACE_dTWS = dTWS(ne_GRACE)
nw_GRACE_dTWS = dTWS(nw_GRACE)
se_GRACE_dTWS = dTWS(se_GRACE)
sw_GRACE_dTWS = dTWS(sw_GRACE)
MDB_GRACE_dTWS = dTWS(MDB_GRACE)

In [13]:
# Insert full time range

full_time = pd.date_range(start='2002-05-15', end='2024-04-15', freq='MS') + pd.Timedelta(days=14)

def insert_time (GRACE_dTWS):
    GRACE_dTWS = GRACE_dTWS.reindex(time=full_time)
    return GRACE_dTWS


ne_GRACE_dTWS = insert_time(ne_GRACE_dTWS)
nw_GRACE_dTWS = insert_time(nw_GRACE_dTWS)
se_GRACE_dTWS = insert_time(se_GRACE_dTWS)
sw_GRACE_dTWS = insert_time(sw_GRACE_dTWS)
MDB_GRACE_dTWS = insert_time(MDB_GRACE_dTWS)

In [16]:
ne_GRACE_dTWS.name = "GRACE_dTWS"
nw_GRACE_dTWS.name = "GRACE_dTWS"
se_GRACE_dTWS.name = "GRACE_dTWS"
sw_GRACE_dTWS.name = "GRACE_dTWS"
MDB_GRACE_dTWS.name = "GRACE_dTWS"

In [17]:
ne_GRACE_dTWS.to_netcdf('Processed Data/GRACE/ne.nc')
nw_GRACE_dTWS.to_netcdf('Processed Data/GRACE/nw.nc')
se_GRACE_dTWS.to_netcdf('Processed Data/GRACE/se.nc')
sw_GRACE_dTWS.to_netcdf('Processed Data/GRACE/sw.nc')
MDB_GRACE_dTWS.to_netcdf('Processed Data/GRACE/MDB.nc')

In [20]:
sw_GRACE_dTWS

<xarray.DataArray (time: 263)> Size: 2kB
array([ 2.09502062e-02,  1.98405291e-02,  2.21964271e-02,  2.60578972e-02,
       -5.52282176e-02, -1.17173082e-02,  3.74396489e-03,  1.48969704e-02,
       -3.82253392e-02,  2.54135255e-02,  1.01946006e-02,  6.80768172e-03,
       -4.16277578e-03, -4.24881245e-03,  4.45326591e-02,  2.74967201e-04,
       -2.77471608e-02, -2.13432941e-02, -4.99651246e-03,  5.36464239e-03,
       -1.29948117e-03,  1.32565718e-03, -4.97598383e-03, -1.70839388e-02,
        2.32526634e-02, -8.73951169e-03,  2.67147869e-02,  2.82711255e-02,
       -3.08995715e-02, -3.72928853e-02,  2.47527699e-02, -1.23878514e-02,
       -5.54067750e-03, -1.00373110e-02, -1.68066490e-04,  7.58592358e-03,
        1.76732492e-02,  2.48557235e-02, -3.34099924e-03, -1.14796975e-02,
       -3.02522043e-03,  7.72599264e-03, -2.48808690e-02, -2.54310594e-02,
       -9.20451317e-03,  5.57652638e-03,  1.65129439e-02,  1.49303518e-02,
       -2.20577536e-02,  1.69011205e-03,  5.26441052e-03,  2.91698254e-02,
       -9.07773699e-03, -2.26984616e-02, -3.31399549e-02, -1.93328118e-02,
        1.22202304e-02,  3.66410301e-02,  1.12582926e-02, -7.67559122e-03,
       -4.10707780e-03,  2.90441619e-02,  6.51838063e-05,  1.72788302e-03,
       -1.02014002e-02, -2.20122322e-02, -5.41117799e-03, -5.23408137e-03,
        1.97132592e-02, -3.61562264e-02, -5.96318127e-03,  2.88798310e-02,
        5.13924820e-03, -3.79788725e-03,  1.63245508e-02, -1.35807980e-02,
       -1.12101864e-02, -1.75203159e-03,  5.48277204e-03, -1.86173153e-02,
...
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan, -1.50556823e-02, -1.50309569e-02, -1.14435891e-02,
       -9.37446366e-03, -5.79934641e-03,  2.49392637e-02, -1.73556702e-02,
       -2.47207270e-02, -2.25021366e-03,  2.79028098e-02, -2.49349619e-04,
        2.48743566e-02, -1.29537044e-02, -1.45578868e-02,  1.83780583e-02,
       -1.91331024e-02,  9.48968050e-03, -3.10712951e-02, -4.87284619e-03,
       -7.83068964e-03,  2.31169705e-02,  3.30818602e-02,  2.00895120e-02,
       -2.11260935e-02,  8.06913454e-03,  3.37363146e-03,  1.07193240e-02,
        2.03693262e-02, -1.48067830e-02, -3.98690762e-02, -9.56714642e-03,
        2.36106916e-02, -5.48217882e-03,  2.22313157e-02, -1.80058065e-02,
        7.52117728e-03,  1.62137208e-02,  2.18251932e-02,  6.07790173e-03,
       -2.85955465e-02, -3.91302107e-02,  2.33588678e-02,  8.10890985e-03,
        1.42952448e-02, -1.03170029e-02, -2.58982576e-02,  3.13983621e-02,
        1.51874000e-02,  3.15417664e-02, -3.62347269e-02,  2.36201588e-02,
        4.37900834e-02,  1.94993812e-02, -3.51473824e-02, -3.89774820e-02,
       -6.56069279e-03, -2.92859566e-02,  4.67932210e-02,  2.06837585e-02,
       -1.79019223e-02,  3.20817127e-02, -3.31418188e-02,  1.95683755e-02,
       -2.38911440e-02,  1.29273784e-02, -2.45465702e-02,  1.17537446e-02,
       -2.24691683e-03, -9.51279331e-03,  1.06022523e-02])
Coordinates:
  * time         (time) datetime64[ns] 2kB 2002-06-15 2002-07-15 ... 2024-04-15
    spatial_ref  int32 4B 0

## Load datasets

In [18]:
precip_data_ERA5=xarray.open_dataset('Processed Data/Models/ERA5-p.nc')
evap_data_ERA5=xarray.open_dataset('Processed Data/Models/ERA5-et.nc')
precip_data_awra=xarray.open_dataset('Processed Data/Models/AWRA-p.nc')
et_data_awra=xarray.open_dataset('Processed Data/Models/AWRA-et.nc')
et_data_cmrset=xarray.open_dataset('Processed Data/Models/CMRSET.nc')

In [19]:
# Data opens as dataset, convert to dataarray
precip_data_ERA5=precip_data_ERA5["tp"]
evap_data_ERA5=evap_data_ERA5["e"]
precip_data_awra=precip_data_awra["rain_day"]
et_data_awra=et_data_awra["etot"]
et_data_cmrset=et_data_cmrset["aET"]

## ERA5

In [20]:
# Process datasets
ne_ERA5_p = functions.process_data(precip_data_ERA5, MDB_northeast_shape)
nw_ERA5_p = functions.process_data(precip_data_ERA5, MDB_northwest_shape)
se_ERA5_p = functions.process_data(precip_data_ERA5, MDB_southeast_shape)
sw_ERA5_p = functions.process_data(precip_data_ERA5, MDB_southwest_shape)
MDB_ERA5_p = functions.process_data(precip_data_ERA5, MDB_shape)

ne_ERA5_et = functions.process_data(evap_data_ERA5, MDB_northeast_shape)
nw_ERA5_et = functions.process_data(evap_data_ERA5, MDB_northwest_shape)
se_ERA5_et = functions.process_data(evap_data_ERA5, MDB_southeast_shape)
sw_ERA5_et = functions.process_data(evap_data_ERA5, MDB_southwest_shape)
MDB_ERA5_et = functions.process_data(evap_data_ERA5, MDB_shape)


In [21]:
# Create xarrays

ne_ERA5_p = create_xarray(ne_ERA5_p,"ERA5_p")
nw_ERA5_p = create_xarray(nw_ERA5_p,"ERA5_p")
se_ERA5_p = create_xarray(se_ERA5_p,"ERA5_p")
sw_ERA5_p = create_xarray(sw_ERA5_p,"ERA5_p")
MDB_ERA5_p = create_xarray(MDB_ERA5_p,"ERA5_p")

ne_ERA5_et = create_xarray(ne_ERA5_et,"ERA5_et")
nw_ERA5_et = create_xarray(nw_ERA5_et,"ERA5_et")
se_ERA5_et = create_xarray(se_ERA5_et,"ERA5_et")
sw_ERA5_et = create_xarray(sw_ERA5_et,"ERA5_et")
MDB_ERA5_et = create_xarray(MDB_ERA5_et,"ERA5_et")

## AWRA

In [22]:
# Process datasets
ne_AWRA_p = functions.process_data(precip_data_awra, MDB_northeast_shape)
nw_AWRA_p = functions.process_data(precip_data_awra, MDB_northwest_shape)
se_AWRA_p = functions.process_data(precip_data_awra, MDB_southeast_shape)
sw_AWRA_p = functions.process_data(precip_data_awra, MDB_southwest_shape)
MDB_AWRA_p = functions.process_data(precip_data_awra, MDB_shape)

ne_AWRA_et = functions.process_data(et_data_awra, MDB_northeast_shape)
nw_AWRA_et = functions.process_data(et_data_awra, MDB_northwest_shape)
se_AWRA_et = functions.process_data(et_data_awra, MDB_southeast_shape)
sw_AWRA_et = functions.process_data(et_data_awra, MDB_southwest_shape)
MDB_AWRA_et = functions.process_data(et_data_awra, MDB_shape)

In [23]:
# Create xarrays

ne_AWRA_p = create_xarray(ne_AWRA_p,"AWRA_p")
nw_AWRA_p = create_xarray(nw_AWRA_p,"AWRA_p")
se_AWRA_p = create_xarray(se_AWRA_p,"AWRA_p")
sw_AWRA_p = create_xarray(sw_AWRA_p,"AWRA_p")
MDB_AWRA_p = create_xarray(MDB_AWRA_p,"AWRA_p")

ne_AWRA_et = create_xarray(ne_AWRA_et,"AWRA_et")
nw_AWRA_et = create_xarray(nw_AWRA_et,"AWRA_et")
se_AWRA_et = create_xarray(se_AWRA_et,"AWRA_et")
sw_AWRA_et = create_xarray(sw_AWRA_et,"AWRA_et")
MDB_AWRA_et = create_xarray(MDB_AWRA_et,"AWRA_et")

## CMRSET

In [24]:
# Process datasets
ne_CMRSET_et = functions.process_data(et_data_cmrset, MDB_northeast_shape)
nw_CMRSET_et = functions.process_data(et_data_cmrset, MDB_northwest_shape)
se_CMRSET_et = functions.process_data(et_data_cmrset, MDB_southeast_shape)
sw_CMRSET_et = functions.process_data(et_data_cmrset, MDB_southwest_shape)
MDB_CMRSET_et = functions.process_data(et_data_cmrset, MDB_shape)

In [25]:
# Add 3 NaN (Because CMRSET is missing last 3 datapoints)
ne_CMRSET_et = np.append(ne_CMRSET_et, [np.nan] * 3)
nw_CMRSET_et = np.append(nw_CMRSET_et, [np.nan] * 3)
se_CMRSET_et = np.append(se_CMRSET_et, [np.nan] * 3)
sw_CMRSET_et = np.append(sw_CMRSET_et, [np.nan] * 3)
MDB_CMRSET_et = np.append(MDB_CMRSET_et, [np.nan] * 3)

In [26]:
# Create xarrays

ne_CMRSET_et = create_xarray(ne_CMRSET_et,"CMRSET_et")
nw_CMRSET_et = create_xarray(nw_CMRSET_et,"CMRSET_et")
se_CMRSET_et = create_xarray(se_CMRSET_et,"CMRSET_et")
sw_CMRSET_et = create_xarray(sw_CMRSET_et,"CMRSET_et")
MDB_CMRSET_et = create_xarray(MDB_CMRSET_et,"CMRSET_et")

## MODIS

In [27]:
# load MODIS data separately
# delete first three months (march-may 2002) and last month (may 2024)
ne_modis_et_mean = np.load("MODIS_ET/ne_modis_et_mean.npy").tolist()[3:-1]
nw_modis_et_mean = np.load("MODIS_ET/nw_modis_et_mean.npy").tolist()[3:-1]
se_modis_et_mean = np.load("MODIS_ET/se_modis_et_mean.npy").tolist()[3:-1]
sw_modis_et_mean = np.load("MODIS_ET/sw_modis_et_mean.npy").tolist()[3:-1]

In [28]:
# Generate list of dates on the 15th of each month
modis_dates = pd.date_range(start='2002-04-15', end='2024-04-15', freq='MS') + pd.Timedelta(days=14)

In [29]:
modis_et_dict = {
    date: [v1, v2, v3, v4]
    for date, v1, v2, v3, v4 in zip(modis_dates, ne_modis_et_mean, nw_modis_et_mean, se_modis_et_mean, sw_modis_et_mean)
}

In [30]:
# Define main time range
start_date = datetime(2002, 5, 1)
end_date = datetime(2024, 4, 30)

# Filter the dictionary
modis_et_dict = {
    date: values
    for date, values in modis_et_dict.items()

}

In [31]:
ne_MODIS_et = np.array([entry[0] for entry in modis_et_dict.values()])/1000 # in m
nw_MODIS_et = np.array([entry[1] for entry in modis_et_dict.values()])/1000
se_MODIS_et = np.array([entry[2] for entry in modis_et_dict.values()])/1000
sw_MODIS_et = np.array([entry[3] for entry in modis_et_dict.values()])/1000 
#MDB_MODIS_et**

In [32]:
# Create xarrays

ne_MODIS_et = create_xarray(ne_MODIS_et,"MODIS_et")
nw_MODIS_et = create_xarray(nw_MODIS_et,"MODIS_et")
se_MODIS_et = create_xarray(se_MODIS_et,"MODIS_et")
sw_MODIS_et = create_xarray(sw_MODIS_et,"MODIS_et")
#MDB_MODIS_et = create_xarray(MDB_MODIS_et,"MODIS_et")

## Runoff

In [38]:
runoff_df = pd.read_csv('Runoff/MDB_runoff.csv', index_col=0, parse_dates=True)

In [40]:
ne_runoff = runoff_df["NE"]
nw_runoff = runoff_df["NW"]
se_runoff = runoff_df["SE"]
sw_runoff = runoff_df["SW"]
MDB_runoff = runoff_df["MDB"]

In [41]:
# Convert to numpy array and flatten
ne_runoff = ne_runoff.values.ravel()
nw_runoff = nw_runoff.values.ravel()
se_runoff = se_runoff.values.ravel()
sw_runoff = sw_runoff.values.ravel()
MDB_runoff = MDB_runoff.values.ravel()

In [42]:
# Create xarrays

ne_runoff = create_xarray(ne_runoff,"runoff")
nw_runoff = create_xarray(nw_runoff,"runoff")
se_runoff = create_xarray(se_runoff,"runoff")
sw_runoff = create_xarray(sw_runoff,"runoff")
MDB_runoff = create_xarray(MDB_runoff,"runoff")

## Combine All xarrays

In [35]:
# Note that ERA5 dataset has negative ET values

In [59]:
combined_data_ne = xarray.Dataset({
    "GRACE_dTWS": ne_GRACE_dTWS,
    "ERA5_p": ne_ERA5_p,
    "ERA5_et": -ne_ERA5_et,
    "AWRA_p": ne_AWRA_p,
    "AWRA_et": ne_AWRA_et,
    "CMRSET_et": ne_CMRSET_et,
    "MODIS_et": ne_MODIS_et,
    "Runoff": ne_runoff
})

In [60]:
combined_data_nw = xarray.Dataset({
    "GRACE_dTWS": nw_GRACE_dTWS,
    "ERA5_p": nw_ERA5_p,
    "ERA5_et": -nw_ERA5_et,
    "AWRA_p": nw_AWRA_p,
    "AWRA_et": nw_AWRA_et,
    "CMRSET_et": nw_CMRSET_et,
    "MODIS_et": nw_MODIS_et,
    "Runoff": nw_runoff
})

In [61]:
combined_data_se = xarray.Dataset({
    "GRACE_dTWS": se_GRACE_dTWS,
    "ERA5_p": se_ERA5_p,
    "ERA5_et": -se_ERA5_et,
    "AWRA_p": se_AWRA_p,
    "AWRA_et": se_AWRA_et,
    "CMRSET_et": se_CMRSET_et,
    "MODIS_et": se_MODIS_et,
    "Runoff": se_runoff
})

In [62]:
combined_data_sw = xarray.Dataset({
    "GRACE_dTWS": sw_GRACE_dTWS,
    "ERA5_p": sw_ERA5_p,
    "ERA5_et": -sw_ERA5_et,
    "AWRA_p": sw_AWRA_p,
    "AWRA_et": sw_AWRA_et,
    "CMRSET_et": sw_CMRSET_et,
    "MODIS_et": sw_MODIS_et,
    "Runoff": sw_runoff
})

In [40]:
#combined_data_MDB = xarray.Dataset({
#    "GRACE_dTWS": MDB_GRACE_dTWS,
#    "ERA5_p": MDB_ERA5_p,
#    "ERA5_et": -MDB_ERA5_et,
#    "AWRA_p": MDB_AWRA_p,
#    "AWRA_et": MDB_AWRA_et,
#    "CMRSET_et": MDB_CMRSET_et,
#    "MODIS_et": MDB_MODIS_et,
#    "Runoff": MDB_runoff
#})

Remove data from 15-06-17 to 15-06-18 (missing data) + 15-07-18 (first time step of new data)

In [65]:
start_time = np.datetime64('2017-06-15')
end_time = np.datetime64('2018-07-15')

def filter (ds):
    filtered_ds = ds.where((ds.time < start_time) | (ds.time > end_time), drop=True)
    return filtered_ds

In [72]:
combined_data_ne = filter(combined_data_ne)
combined_data_nw = filter(combined_data_nw)
combined_data_se = filter(combined_data_se)
combined_data_sw = filter(combined_data_sw)
#combined_data_MDB = filter(combined_data_MDB)

In [73]:
combined_data_ne.to_netcdf('Processed Data/XArrays/ne_combined.nc')
combined_data_nw.to_netcdf('Processed Data/XArrays/nw_combined.nc')
combined_data_se.to_netcdf('Processed Data/XArrays/se_combined.nc')
combined_data_sw.to_netcdf('Processed Data/XArrays/sw_combined.nc')
#combined_data_MDB.to_netcdf('Processed Data/XArrays/MDB_combined.nc')